In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

In [2]:
def get_data(year, url):
    api = "Bearer INSERT_KEY_HERE"
    header = {'Authorization':api}
    params = {'year':year}


    response = requests.get(f"http://api.collegefootballdata.com/{url}",
                            headers=header,
                            params=params)

    response_json = response.json()

    recruiting = pd.json_normalize(response_json)
    
    return recruiting

In [3]:
BEGIN_YEAR = 2011
END_YEAR = 2023

In [4]:
# IRS income dataset
income = pd.read_csv("INSERT_INCOME_FILE_HERE.csv")

# Coordinates with zip codes
zips_coordinates = pd.read_csv("zip_code_database.csv")

zips_coordinates_og = zips_coordinates.copy()

# replace 
cfb = pd.concat([get_data(year, "recruiting/players") for year in range(BEGIN_YEAR, END_YEAR, 1)])

In [5]:
# data for win%
cfb_results = pd.concat([get_data(year, "records") for year in range(BEGIN_YEAR, END_YEAR + 2)])

In [6]:
# data for productivity metrics
cfb_player_ppa = pd.concat([get_data(year, "ppa/players/season") for year in range(BEGIN_YEAR, END_YEAR + 2)])

In [7]:
position_dict = {
    'RET' : "returner",
    "FB" : "full back",
    "LS" : "long snapper",
    "EDGE" : "edge",
    "QB" : "quarterback",
    "IOL" : "interior offensive line",
    "P" : "punter",
    "LB" : "linebacker",
    "DL" : "defensive lineman",
    "K" : "kicker",
    "APB" : "all purpose back",
    "OC" : "center",
    "DUAL" : "quarterback",
    "PRO" : "quarterback",
    "WDE" : "weak side defensive end",
    "ILB" : "inside linebacker",
    "SDE" : "strong side defensive end", 
    "TE" : "tight end",
    "OG" : "offensive guard",
    "DT" : "defensive tackle",
    "OLB" : "outside linebacker",
    "RB" : "running back",
    "S" : "safety",
    "ATH" : "athlete",
    "OT" : "offensive tackle",
    "CB" : "cornerback",
    "WR" : "wide receiver"
}

In [8]:
special_teams = ['RET', 'LS', 'P', 'K']
defensive_line = ['EDGE', 'DL', 'WDE', 'SDE', 'DT']
linebackers = ["LB", "ILB", "OLB"]
secondary = ["CB", "S"]
backs = ['FB', "APB", "RB"]
receivers = ["WR", "TE"]
offensive_line = ["IOL", "OC", "OG", "OT"]
other = ["ATH"]
qb = ['QB', 'DUAL', 'PRO']

In [9]:
def assign_position_group(position):
    if position in special_teams:
        return "special teams"
    
    elif position in defensive_line:
        return "defensive line"
    
    elif position in linebackers:
        return "linebackers"
    
    elif position in secondary:
        return "secondary"
    
    elif position in backs:
        return "backs"
    
    elif position in receivers:
        return "receiver"
    
    elif position in offensive_line:
        return "offensive line"
    
    elif position in qb:
        return "quarterback"
    
    else:
        return "other"

In [10]:
cfb['position'] = cfb['position'].apply(lambda x: assign_position_group(x))

In [11]:
# removing schools with little representation
cfb = cfb[cfb['committedTo'].isin(cfb.groupby("committedTo").size().sort_values().head(100).index) == False]

In [12]:
# isolating and removing non-US recruits
non_us_players = [cfb['hometownInfo.latitude'].isna()]
cfb = cfb[cfb['hometownInfo.latitude'].isna() == False]

In [13]:
# slimming down dfs for visual ease
relevant_columns_income = ['STATE', 'zipcode', 'A02650']

relevant_columns_linking_info = ['state', 'zip', 'primary_city', 'latitude', 'longitude']

income = income[relevant_columns_income]

zips_coordinates = zips_coordinates[relevant_columns_linking_info]

In [14]:
# coordinates with zip
zips_coordinates.head()

,state,zip,primary_city,latitude,longitude
0,NY,501,Holtsville,40.81,-73.04
1,NY,544,Holtsville,40.81,-73.04
2,PR,601,Adjuntas,18.16,-66.72
3,PR,602,Aguada,18.38,-67.18
4,PR,603,Aguadilla,18.43,-67.15


In [15]:
# income info
income.head()

,STATE,zipcode,A02650
0,AL,0,11036309.0
1,AL,0,17960153.0
2,AL,0,16216787.0
3,AL,0,14588959.0
4,AL,0,28985528.0


In [16]:
# avg income by zipcode
median_income_by_zipcode = income.merge(zips_coordinates, left_on=['zipcode'], right_on=['zip']).groupby("zipcode").A02650.median().reset_index()


In [17]:
# merging with other info
merged_income_zipcor = zips_coordinates.merge(median_income_by_zipcode, left_on=['zip'], right_on=['zipcode'])

In [18]:
# adjusting coordinates for merging
merged_income_zipcor['adj_lat'] = merged_income_zipcor['latitude'].apply(lambda x: float(str(x).split(".")[0]))
merged_income_zipcor['adj_lng'] = merged_income_zipcor['longitude'].apply(lambda x: float(str(x).split(".")[0]))

In [19]:
# adjusting coordinates for merging
zips_coordinates_og['adj_lat'] = zips_coordinates_og['latitude'].apply(lambda x: float(str(x).split(".")[0]))
zips_coordinates_og['adj_lng'] = zips_coordinates_og['longitude'].apply(lambda x: float(str(x).split(".")[0]))

In [20]:
# adjusting coordinates for merging
cfb['lat_adj'] = cfb['hometownInfo.latitude'].apply(lambda x: float(str(x).split(".")[0]))
cfb['lng_adj'] = cfb['hometownInfo.longitude'].apply(lambda x: float(str(x).split(".")[0]))

In [21]:
# getting average income by city
income_by_city_state_coord = merged_income_zipcor.groupby(['primary_city', 'state', 'adj_lat', 'adj_lng']).A02650.median().reset_index()
income_by_city_state_coord['A02650'] = income_by_city_state_coord['A02650'].round()
income_by_city_state_coord = income_by_city_state_coord.rename(columns={"A02650" : "average_income"})


In [22]:
# getting average income by city **** FOR DISPLAYING DIFFERENCE IN USING ADJ COORD AND NOT *****
income_by_city_state_coord_ = merged_income_zipcor.groupby(['primary_city', 'state']).A02650.median().reset_index()
income_by_city_state_coord_['A02650'] = income_by_city_state_coord_['A02650'].round()
income_by_city_state_coord_ = income_by_city_state_coord_.rename(columns={"A02650" : "average_income"})


In [23]:
# joining together for final df
final = cfb.merge(income_by_city_state_coord, left_on=['city', 'stateProvince', 'lat_adj', 'lng_adj'],
                                       right_on=['primary_city', 'state', 'adj_lat', 'adj_lng'])

In [24]:
# handling duplicates
final = final.groupby(['name', 'committedTo']).first().reset_index()

In [25]:
# removing unranked
final = final[final['ranking'].isna() == False]

In [26]:
# removing those with missing info
final = final[(final['height'].isna() == False) & (final['weight'].isna() == False)]

In [27]:
# final dupe check
final = final[final[['name', 'hometownInfo.longitude']].duplicated() == False]

In [28]:
# new column for overall ranking across all years for players
final['entire_dataset_ranking'] = final['rating'].rank(ascending=False)

# Linking with team results

In [29]:
cfb_results['win%'] = cfb_results['total.wins'] / cfb_results['total.games']

In [30]:
final_with_results = final.merge(cfb_results[['team', 'year', 'conference', 'division', 'win%']],
            left_on=['committedTo', 'year'], right_on=['team', 'year'], how='left')

In [31]:
final_with_results = final_with_results[final_with_results['win%'].isna() == False]

In [32]:
# agg correlation results
final_with_results.groupby("team")[['win%', "rating", 'stars']].mean().corr()

,win%,rating,stars
win%,1.000000,0.586174,0.586701
rating,0.586174,1.000000,0.996709
stars,0.586701,0.996709,1.000000


In [33]:
# getting correlations by position
outcome_corr_by_position = final_with_results[['position', 'win%', 'rating', 'average_income', 'stars']].groupby("position").corr().reset_index()


In [34]:
qbs_and_dline = outcome_corr_by_position[outcome_corr_by_position['position'].isin(['quarterback', 'defensive line'])]

In [35]:
# correlation for positions
outcome_corr_by_position[outcome_corr_by_position['level_1'] == 'stars'].sort_values(by='win%')\
.rename(columns={"level_1" : "variable"})[['position', 'variable', 'win%']]

,position,variable,win%
35,special teams,stars,0.178222
23,quarterback,stars,0.217516
19,other,stars,0.223605
27,receiver,stars,0.250262
31,secondary,stars,0.260744
11,linebackers,stars,0.267615
15,offensive line,stars,0.275081
3,backs,stars,0.277587
7,defensive line,stars,0.284801


In [36]:
qbs_and_dline.set_index(["position", "level_1"])

win%    rating  average_income     stars
position       level_1                                                     
defensive line win%            1.000000  0.320985       -0.009984  0.284801
               rating          0.320985  1.000000        0.001948  0.913571
               average_income -0.009984  0.001948        1.000000  0.000222
               stars           0.284801  0.913571        0.000222  1.000000
quarterback    win%            1.000000  0.240224       -0.017400  0.217516
               rating          0.240224  1.000000        0.056378  0.900505
               average_income -0.017400  0.056378        1.000000  0.056859
               stars           0.217516  0.900505        0.056859  1.000000

# Linking with player productivity metrics

In [37]:
# making sure players have played more than 50 snaps
cfb_player_ppa = cfb_player_ppa[cfb_player_ppa['countablePlays'] > 50]

In [38]:
# filling na with 0
cfb_player_ppa.fillna(0.0, inplace=True)

In [39]:
# getting average productivity by player
productivity_by_player = cfb_player_ppa.groupby(["name", "team", "position"]).mean().reset_index()

In [40]:
# linking with recruiting info
ppa_and_recruiting = cfb[['year', 'ranking', 'height', 'weight',
                              'stars', 'rating', 'name', 'committedTo']].merge(productivity_by_player, left_on=['name', 'committedTo'], right_on=['name', 'team'])

In [41]:
# focuing on QBs
qbs_ppa = ppa_and_recruiting[ppa_and_recruiting['position'] == 'QB'].drop(['year', 'ranking', 'height', 'weight', 'season'], axis=1)

In [42]:
# creating correlation matrix
qb_ppa_corr = qbs_ppa.corr()[['stars']].reset_index().rename(columns={'index' : "variable"})

In [43]:
qb_ppa_corr

,variable,stars
0,stars,1.000000
1,rating,0.912002
2,countablePlays,0.133904
3,averagePPA.all,0.182831
4,averagePPA.pass,0.162307
5,averagePPA.rush,0.051654
6,averagePPA.firstDown,0.178390
7,averagePPA.secondDown,0.146767
8,averagePPA.thirdDown,0.087869
9,averagePPA.standardDowns,0.171149
